# 0. import libraries

In [1]:
# import libraries
import json
import re # regular expression
import gc # to collect memory
from datetime import datetime

import tensorflow as tf
import pandas as pd
import numpy as np

# for NLP
# from konlpy.tag import Kkma
# from konlpy.utils import pprint
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
# import missingno as msno

plt.style.use('seaborn')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
font_name = matplotlib.font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
matplotlib.rc('font', family = font_name)

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

---
# 1. data

In [9]:
# load json files
with open("./res/genre_gn_all.json", encoding = 'utf-8') as json_file: genre_gn_all = json.load(json_file)
with open("./res/song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
with open("./res/train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
with open("./res/val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)

# json to dataframe
genre_gn_all = pd.Series(genre_gn_all).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
song_meta = pd.DataFrame(song_meta)
train = pd.DataFrame(train)
valid = pd.DataFrame(valid)

In [12]:
#대분류장르코드
# 장르코드 뒷자리 두 자리가 00인 코드를 필터링
gnr_code = genre_gn_all[genre_gn_all['gen_id'].str[-2:] == '00']

In [17]:
#상세장르코드
# 장르코드 뒷자리 두 자리가 00이 아닌 코드를 필터링
dtl_gnr_code = genre_gn_all[genre_gn_all['gen_id'].str[-2:] != '00']
dtl_gnr_code.rename(columns = {'gen_id' : 'dtl_gen_id', 'genre' : 'dtl_genre'}, inplace = True)

In [3]:
df_train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


---
# 목표
### 대분류에서 장르명과 일치하는 단어가 있는 태그를 찾는다.
---

---
## 1) tag dataframe 제작

In [20]:
# 플레이리스트 아이디(id)와 매핑된 태그(tags) 추출
plylst_tag_map = train[['id', 'tags']]

# unnest tags
plylst_tag_map_unnest = np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
        np.concatenate(plylst_tag_map.tags.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
plylst_tag_map = pd.DataFrame(data = plylst_tag_map_unnest[0], columns = plylst_tag_map.columns)
plylst_tag_map['id'] = plylst_tag_map['id'].astype(str)

# unnest 객체 제거
del plylst_tag_map_unnest

In [21]:
plylst_tag_map 


,id,tags
0,61281,락
1,10532,추억
2,10532,회상
3,76951,까페
4,76951,잔잔한
...,...,...
476326,131982,퇴근길
476327,100389,노래추천
476328,100389,팝송추천
476329,100389,팝송


### 이건 그냥 플레이리스트 아이디와 맵핑된 코드였음.. 416331개가 아님

In [22]:
# 태그 별 매핑 빈도 수 저장 
tag_cnt = plylst_tag_map.groupby('tags').tags.count().reset_index(name = 'mapping_cnt')
tag_cnt['tags'] = tag_cnt['tags'].astype(str)
tag_cnt['mapping_cnt'] = tag_cnt['mapping_cnt'].astype(int)

In [42]:
tag_cnt.head(100)

,tags,mapping_cnt
0,,1
1,00,4
2,007,3
3,007시리즈,2
4,00s,3
5,00년,6
6,00년대,27
7,00년대노래,1
8,00년대발라드,2
9,00년도노래,1


In [46]:
tag_cnt.tail(100)

,tags,mapping_cnt
29060,힙플,2
29061,힙플라디오,1
29062,힙플레이스,1
29063,힙플페,3
29064,힙하게,1
29065,힙하고,1
29066,힙하고싶은날,1
29067,힙하다,4
29068,힙하우스,2
29069,힙한,209


In [43]:
tag_cnt

,tags,mapping_cnt
0,,1
1,00,4
2,007,3
3,007시리즈,2
4,00s,3
...,...,...
29155,힙해,5
29156,힙힙힙,2
29157,힛뎀포크,1
29158,힛뎀폭,1


### 태그 자체는 29160개였다!
전체 태그 저장해놓은 df: tag_cnt

---

## 2) 대분류에서 장르 name dataframe 만들기
- 유의사항
  - 중복 있음
  - 랩/힙합과 같이 /로 나눠진 것 존재
  - pop의 경우 팝이라는 단어로 더 많이 쓰임
---
- 중복 제거 필요
- / 있는 경우 나눠서 넣음
- 알앤비, 소울, rock, metal, 팝, country, 등등 단어 추가 (일단 생각나는게 이정도라서...)

In [189]:
gnr_code

,gen_id,genre
0,GN0100,발라드
6,GN0200,댄스
12,GN0300,랩/힙합
18,GN0400,R&B/Soul
22,GN0500,인디음악
32,GN0600,록/메탈
39,GN0700,성인가요
50,GN0800,포크/블루스
56,GN0900,POP
65,GN1000,록/메탈


In [362]:
genre_list=gnr_code[['genre']]

In [363]:
genre_list

,genre
0,발라드
6,댄스
12,랩/힙합
18,R&B/Soul
22,인디음악
32,록/메탈
39,성인가요
50,포크/블루스
56,POP
65,록/메탈


In [364]:
var_split = genre_list['genre'].str.split('/')
var_split


#value_name = var_split.str[0]

#2,3번째 항목에서 NaN 제거
value_name1 = var_split.str[1]
value_name1=value_name1[~value_name1.isnull()]
value_name2 = var_split.str[2]
value_name2=value_name2[~value_name2.isnull()]

#장르 리스트에서 /없애고, /로 나눈 항목 중 첫번째 것만 넣음
genre_list['genre'] = genre_list['genre'].str.split('/').str[0]

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [365]:
#object를 dataframe로
value_name1=pd.DataFrame(value_name1)
value_name1

,genre
12,힙합
18,Soul
32,메탈
50,블루스
65,메탈
90,힙합
100,Soul
109,블루스
198,태교


In [366]:
value_name2=pd.DataFrame(value_name2)
value_name2

,genre
109,컨트리


In [367]:
genre_list=pd.concat([genre_list, value_name1,value_name2], ignore_index=True)
genre_list

,genre
0,발라드
1,댄스
2,랩
3,R&B
4,인디음악
5,록
6,성인가요
7,포크
8,POP
9,록


In [368]:
genre_list=genre_list.drop_duplicates() #중복 제거

#index reset하겠음(문자 단위 검색 목적이므로)
genre_list=genre_list.reset_index()
del genre_list['index']

genre_list

,genre
0,발라드
1,댄스
2,랩
3,R&B
4,인디음악
5,록
6,성인가요
7,포크
8,POP
9,일렉트로니카


In [369]:
#키워드 추가
#dance, 알앤비, rock, 인디, 성인, 팝, 일렉트로닉, 키즈, 종교, idol, 소울, 일본(12개)
genre_list=genre_list.append([{'genre':'dance'},{'genre':'알앤비'},{'genre':'rock'},{'genre':'인디'},{'genre':'성인'},
                           {'genre':'팝'},{'genre':'일렉트로닉'},{'genre':'키즈'},{'genre':'종교'},{'genre':'idol'},
                              {'genre':'소울'},{'genre':'일본'}]
                          ,ignore_index=True)
genre_list

,genre
0,발라드
1,댄스
2,랩
3,R&B
4,인디음악
5,록
6,성인가요
7,포크
8,POP
9,일렉트로니카


---

## 3) 장르와 연관된 태그
tag_cnt: 태그 모음  
genre_list: 장르 모음

- 문자열 검색시 contains는 일부 포함된것도 검색
- isin은 완벽하게 일치인듯?

In [370]:
#'발라드'
print(genre_list['genre'][0])
test=tag_cnt[tag_cnt['tags'].str.contains(genre_list['genre'][0])] 
test=test.reset_index()
del test['index']
test

발라드


,tags,mapping_cnt
0,00년대발라드,2
1,1980년대발라드,1
2,1990년대발라드,2
3,2000년대발라드,6
4,7080발라드,1
...,...,...
114,한국발라드,6
115,한국형발라드,1
116,혼성발라드,1
117,힙발라드,4


In [371]:
#'댄스'
print(genre_list['genre'][1])
test=tag_cnt[tag_cnt['tags'].str.contains(genre_list['genre'][1])] 
test=test.reset_index()
del test['index']
test

댄스


,tags,mapping_cnt
0,2010년대댄스,1
1,30대댄스가요,1
2,8090댄스,1
3,80년대댄스팝,1
4,90년대_댄스,1
...,...,...
135,하우스_댄스,1
136,헬스장에서듣는댄스곡,1
137,혼성댄스그룹,1
138,힐링댄스,1


### 전체 출력(반복문)

In [376]:
for i in range(0,43):
    print("index %d: " %i,end='')
    print(genre_list['genre'][i]+'\n')
    test=tag_cnt[tag_cnt['tags'].str.contains(genre_list['genre'][i])] 
    test=test.reset_index()
    del test['index']
    print(test)
    print("\n---------------------------------------------------------\n")

index 0: 발라드

          tags  mapping_cnt
0      00년대발라드            2
1    1980년대발라드            1
2    1990년대발라드            2
3    2000년대발라드            6
4      7080발라드            1
..         ...          ...
114      한국발라드            6
115     한국형발라드            1
116      혼성발라드            1
117       힙발라드            4
118      힙합발라드            1

[119 rows x 2 columns]

---------------------------------------------------------

index 1: 댄스

           tags  mapping_cnt
0      2010년대댄스            1
1       30대댄스가요            1
2        8090댄스            1
3       80년대댄스팝            1
4       90년대_댄스            1
..          ...          ...
135      하우스_댄스            1
136  헬스장에서듣는댄스곡            1
137      혼성댄스그룹            1
138        힐링댄스            1
139        힙합댄스            4

[140 rows x 2 columns]

---------------------------------------------------------

index 2: 랩

         tags  mapping_cnt
0        CCM랩            1
1         감성랩           14
2       감성적인랩            1
3

    tags  mapping_cnt
0  뮤직테라피           12

---------------------------------------------------------

index 23: 뮤지컬

       tags  mapping_cnt
0     국외뮤지컬            3
1       뮤지컬           84
2    뮤지컬OST           12
3   뮤지컬_OST            1
4     뮤지컬넘버           14
5     뮤지컬노래            7
6     뮤지컬들의            1
7     뮤지컬배우            1
8     뮤지컬영화            8
9     뮤지컬음악            3
10   뮤지컬풍노래            1
11    초기뮤지컬            1
12  하이스쿨뮤지컬            1
13    해외뮤지컬            2

---------------------------------------------------------

index 24: 크리스마스

            tags  mapping_cnt
0   더매직오브크리스마스타임            1
1       라스트크리스마스            1
2       로맨틱크리스마스            1
3        메리크리스마스           50
4      미리듣는크리스마스            2
5      미리메리크리스마스            9
6        솔로크리스마스            1
7       아이돌크리스마스            1
8        여름크리스마스            1
9     올해도메리크리스마스            1
10     외로운_크리스마스            1
11     재즈보컬크리스마스            1
12      재즈와크리스마스            1
13       

             tags  mapping_cnt
0        2010년대인디            2
1    2월Yasisi인디뮤직            1
2             k인디            1
3        가을과인디의만남            1
4            가을인디            1
..            ...          ...
124        케이인디차트           17
125          한국인디            8
126          해외인디            9
127        해외인디음악            1
128       흔하지않은인디            1

[129 rows x 2 columns]

---------------------------------------------------------

index 35: 성인

          tags  mapping_cnt
0          감성인            1
1         감성인디           39
2           성인            8
3         성인가요           85
4   성인가요Top100            1
5      성인가요스테디            1
6        성인발라드            1
7          성인식            3
8         성인인증            1
9        성인자장가            1
10      슬픈성인가요            1
11        여성인디            2

---------------------------------------------------------

index 36: 팝

         tags  mapping_cnt
0     1970년대팝            2
1     1980년대팝            4
2      1997팝